In [ ]:

from IPython.display import clear_output

from ipaddress import ip_address
import ipywidgets as widgets
import pandas as pd
import altair as alt
import pathlib

file_path = pathlib.Path("Task 1\\20240701_19-20_100l.csv")
dir_path = pathlib.Path().resolve()

csv_file = pd.read_csv(dir_path / file_path)
data: pd.DataFrame = pd.DataFrame(csv_file)

apply_changes_widget = widgets.Button(
    description='Apply changes',
    tooltip='Apply changes',
    disabled=False,
)

top_n_widget = widgets.Dropdown(
    options=['10', '20', '30', '40'],
    value='10',
    description='Top N stats',
    disabled=False,
)

src_dst_widget = widgets.ToggleButtons(
    options=['Received', 'Sent'],
    description='Direction:',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
)

data_type_widget = widgets.ToggleButtons(
    options=['Bytes', 'Packets', 'Flows'],
    description='Data type:',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
)

period_widget = widgets.IntSlider(
    min=0,
    max=60,
    value=30,
    step=5,
    description='Time range in minutes',
    disabled=False,
)

ipv4_start_widget = widgets.Text(
    value='0.0.0.0',
    description='Start IPv4 adress',
    disabled=False,
)

ipv4_end_widget = widgets.Text(
    value='255.255.255.255',
    description='End IPv4 adress',
    disabled=False,
)

ipv6_start_widget = widgets.Text(
    value='0:0:0:0:0:0:0:0',
    description='Start IPv6 adress',
    disabled=False,
)

ipv6_end_widget = widgets.Text(
    value='ffff:ffff:ffff:ffff:ffff:ffff:ffff:ffff',
    description='End IPv6 adress',
    disabled=False,
)

ui_widgets = [
              top_n_widget, src_dst_widget, data_type_widget, 
              period_widget, ipv4_start_widget, ipv4_end_widget, 
              ipv6_start_widget, ipv6_end_widget, apply_changes_widget]

def try_convert(x):
    try:
        return int(ip_address(x))
    except Exception as err:
        return 0

def process_by_type(g_src_dst, g_data_type, top_n_n, curr_data):
    if g_data_type == "FLOWS":
        processed_data = curr_data.groupby(g_src_dst)[g_src_dst].count().reset_index(name='count').nlargest(top_n_n, columns='count')

    else:
        processed_data = curr_data[[g_src_dst, g_data_type]].groupby(g_src_dst).sum().reset_index().nlargest(top_n_n, columns=g_data_type)
    return processed_data

def process_by_ip_range(g_src_dst, ipv4_start_str, ipv4_end_str, ipv6_start_str ,ipv6_end_str):
    ipv4_start_int = int(ip_address(ipv4_start_str))
    ipv4_end_int = int(ip_address(ipv4_end_str))
    ipv6_start_int = int(ip_address(ipv6_start_str))
    ipv6_end_int = int(ip_address(ipv6_end_str))

    return data.loc[
        data[g_src_dst].apply(try_convert).between(ipv4_start_int, ipv4_end_int)
    ]

def get_traffic_data(g_src_dst, g_data_type, top_n, time_period, ipv4_start_str, ipv4_end_str, ipv6_start_str, ipv6_end_str):
    processed_data = process_by_ip_range(g_src_dst, ipv4_start_str, ipv4_end_str, ipv6_start_str, ipv6_end_str)

    print(processed_data[g_src_dst])
    processed_data = process_by_type(g_src_dst, g_data_type, top_n, processed_data)
    
    print(processed_data[g_src_dst])
    return alt.Chart(processed_data).mark_bar().encode(
        x=processed_data.columns[0],
        y=processed_data.columns[1],
    )


def on_change(v):
    clear_output(wait=True)
    
    if src_dst_widget.value == 'Received':
        g_src_dst = "SOURCE IP ADDRESS"
    else:
        g_src_dst = "DESTINATION IP ADDRESS"

    if data_type_widget.value == "Bytes":
        g_data_type = "BYTES"
    elif data_type_widget.value == "Packets":
        g_data_type = "PACKETS"
    elif data_type_widget.value == "Flows":
        g_data_type = "FLOWS"
    
    graph = get_traffic_data(g_src_dst, g_data_type, 
                             int(top_n_widget.value), int(period_widget.value),
                             ipv4_start_widget.value,ipv4_end_widget.value, 
                             ipv6_start_widget.value,ipv6_end_widget.value
                             )

    for ui_widget in ui_widgets:
        display(ui_widget)
    display(graph)

apply_changes_widget.on_click(on_change)
for ui_widget in ui_widgets:
    display(ui_widget)





10        195.113.171.105
12           185.169.4.49
18         206.168.34.101
24          185.16.39.241
29           217.69.97.55
               ...       
143344     193.163.125.82
143399     195.178.87.225
143413      217.69.96.233
143414      211.72.99.242
143425     162.142.125.21
Name: SOURCE IP ADDRESS, Length: 20554, dtype: object
7300    195.113.144.158
7254    195.113.144.116
7294    195.113.144.152
7409     195.113.144.32
7449      195.113.144.7
7267    195.113.144.128
7392    195.113.144.241
7266    195.113.144.127
7320    195.113.144.176
7384    195.113.144.234
Name: SOURCE IP ADDRESS, dtype: object


Dropdown(description='Top N stats', options=('10', '20', '30', '40'), value='10')

ToggleButtons(button_style='info', description='Direction:', options=('Received', 'Sent'), value='Received')

ToggleButtons(button_style='info', description='Data type:', index=2, options=('Bytes', 'Packets', 'Flows'), v…

IntSlider(value=30, description='Time range in minutes', max=60, step=5)

...

Text(value='255.255.255.255', description='End IPv4 adress')

Text(value='0:0:0:0:0:0:0:0', description='Start IPv6 adress')

Text(value='ffff:ffff:ffff:ffff:ffff:ffff:ffff:ffff', description='End IPv6 adress')

Button(description='Apply changes', style=ButtonStyle(), tooltip='Apply changes')

alt.Chart(...)